In [87]:
import pandas as pd

df = pd.read_csv("/home/kyrylo/bucket-72907_mount/football-files/VideoActionTimeStamps.csv")
df

,VidInfo,Action,StartFrame,EndFrame
0,vid1,pass,347,370
1,vid1,goal,583,654
2,vid1,pass,1051,1096
3,vid1,pass,1910,1944
4,vid1,pass,2748,2796
...,...,...,...,...
2552,yt3,pass,67343,67428
2553,yt3,pass,67568,67623
2554,yt3,pass,67950,67977
2555,yt3,pass,70392,70432


In [88]:
df.drop(df[df.Action == 'assist'].index, axis=0, inplace=True)
df

,VidInfo,Action,StartFrame,EndFrame
0,vid1,pass,347,370
1,vid1,goal,583,654
2,vid1,pass,1051,1096
3,vid1,pass,1910,1944
4,vid1,pass,2748,2796
...,...,...,...,...
2552,yt3,pass,67343,67428
2553,yt3,pass,67568,67623
2554,yt3,pass,67950,67977
2555,yt3,pass,70392,70432


In [89]:
df['Len'] = df.EndFrame - df.StartFrame + 1
df

,VidInfo,Action,StartFrame,EndFrame,Len
0,vid1,pass,347,370,24
1,vid1,goal,583,654,72
2,vid1,pass,1051,1096,46
3,vid1,pass,1910,1944,35
4,vid1,pass,2748,2796,49
...,...,...,...,...,...
2552,yt3,pass,67343,67428,86
2553,yt3,pass,67568,67623,56
2554,yt3,pass,67950,67977,28
2555,yt3,pass,70392,70432,41


In [90]:
df.groupby(['Action']).Len.mean()

Action
goal      49.385714
pass      55.620246
save      37.398734
tackle    30.012048
Name: Len, dtype: float64

In [91]:
df.groupby(['Action']).Len.max()

Action
goal      159
pass      211
save      156
tackle    101
Name: Len, dtype: int64

In [70]:
# len(df[df.Len > 100]) -- 120
# df.Action.value_counts() # passes 1500 - tackles x6

In [92]:
rows = []
for vid in df.VidInfo.unique():
    df2 = df[df.VidInfo == vid].copy()
    counter = 0
    for i in range(len(df2)):
        if i < len(df2) - 1:
            if (df2.iloc[i+1].StartFrame - df2.iloc[i].EndFrame > df2[df2.Action == df2.iloc[i].Action].Len.mean() + 11) & (df2.iloc[i+1].VidInfo == df2.iloc[i].VidInfo):
                mean = df2[df2.Action == df2.iloc[i].Action].Len.mean()
                vidname = df2.iloc[i].VidInfo
                startframe = df2.iloc[i].EndFrame+5
                endframe = startframe+round(mean)

                row = [vidname, 'none', startframe, endframe, endframe-startframe]
                rows.append(row)
                if df2.iloc[i+1].StartFrame - endframe > mean + 4:
                    startframe = endframe + 2
                    endframe = startframe + round(mean)
                    row = [vidname, 'none', startframe, endframe, endframe-startframe]
                    rows.append(row)
                    counter += 1
                if df2.iloc[i+1].StartFrame - endframe > mean + 4:
                    startframe = endframe + 2
                    endframe = startframe + round(mean)
                    row = [vidname, 'none', startframe, endframe, endframe-startframe]
                    rows.append(row)
            counter += 1
        if counter >= 155:
            break
len(rows)

1565

In [93]:
df = pd.concat([df, pd.DataFrame(rows, columns=df.columns)], ignore_index=True)
len(df)

4049

In [94]:
df.drop(df[df.Len > 100].index, inplace=True)
len(df)

3913

In [95]:
df.Action.value_counts()

Action
pass      1747
none      1565
tackle     248
goal       199
save       154
Name: count, dtype: int64

In [96]:
import numpy as np

remove_pass = 247
remove_none = 65

drop_pass = list(np.random.choice(df[df.Action == 'pass'].index, remove_pass, replace=False))
drop_none = list(np.random.choice(df[df.Action == 'none'].index, remove_none, replace=False))

# print(len(drop_none.extend(drop_pass)))
for el in drop_none:
    drop_pass.append(el)

df.drop(drop_pass, inplace=True)
df.Action.value_counts()

Action
pass      1500
none      1500
tackle     248
goal       199
save       154
Name: count, dtype: int64

In [97]:
df[df.Action == 'none']

,VidInfo,Action,StartFrame,EndFrame,Len
2484,vid1,none,375,421,46
2485,vid1,none,423,469,46
2486,vid1,none,471,517,46
2487,vid1,none,659,701,42
2488,vid1,none,703,745,42
...,...,...,...,...,...
4044,yt3,none,30684,30707,23
4045,yt3,none,30709,30732,23
4046,yt3,none,31230,31275,45
4047,yt3,none,31277,31322,45


In [98]:
num = 0
for i in df.VidInfo.unique():
    amount = len(df[(df.Action == 'none') & (df.VidInfo == i)])
    print(amount)
    num += amount
print(num)

169
172
194
139
178
152
154
161
181
1500


In [105]:
df.Action.value_counts()

Action
pass      1500
none      1500
tackle     248
goal       199
save       154
Name: count, dtype: int64

In [1]:
import os
num = 0
for folder in os.listdir('/home/kyrylo/dev/clips/clips_dataset'):
    amount = len(os.listdir(f'/home/kyrylo/dev/clips/clips_dataset/{folder}'))
    print(f"{folder}-{amount}")
    num += amount

print(num)

goal-1592
pass-1500
save-1540
tackle-1736
none-1500
7868


In [53]:
import pandas as pd

df = pd.read_csv("actions.csv")
df[(df.Action == 'none')].VidInfo.value_counts()

VidInfo
vid3    194
yt3     181
vid5    178
vid2    172
vid1    169
yt5     161
yt2     154
yt1     152
vid4    139
Name: count, dtype: int64

In [ ]:
# df.to_csv("/home/kyrylo/dev/actions.csv")

In [17]:
import pandas as pd

df = pd.read_csv('actions.csv')
df.Action.value_counts()

Action
pass      1500
none      1500
tackle     248
goal       199
save       154
Name: count, dtype: int64

In [18]:
df.drop(df.columns[0], axis=1, inplace=True)
df.columns

Index(['VidInfo', 'Action', 'StartFrame', 'EndFrame', 'Len'], dtype='object')

In [20]:
df = df[df.Action != 'none']
len(df)

2101

In [25]:
rows = []
for vid in df.VidInfo.unique():
    df2 = df[df.VidInfo == vid].copy()
    
    for i in range(len(df2)):
        if i < len(df2) - 1:
            endframe = df2.iloc[i].EndFrame
            startframe = df2.iloc[i+1].StartFrame
            counter = 0
            while startframe - endframe > 103:
                vidname = df2.iloc[i].VidInfo
                start = endframe + 1
                end = start + 99

                row = [vidname, 'none', start, end, end-start+1]
                rows.append(row)
                counter += 1
                if counter >= 6:
                    break
                
                endframe = end + 2

len(rows)

4889

In [29]:
rows[9]

['vid1', 'none', np.int64(1505), np.int64(1604), np.int64(100)]

In [27]:
df.Action.value_counts()

Action
pass      1500
tackle     248
goal       199
save       154
Name: count, dtype: int64

In [32]:
df2 = pd.DataFrame(rows, columns=['VidInfo', 'Action', 'StartFrame', 'EndFrame', 'Len'])

In [34]:
df2.VidInfo.value_counts()

VidInfo
yt2     1196
yt1      713
yt5      584
vid4     547
vid5     469
yt3      422
vid2     399
vid3     317
vid1     242
Name: count, dtype: int64

In [38]:
import numpy as np

leave = 167
# remove_yt2 = 247
# remove_none = 65
for vid in df2.VidInfo.unique():
    to_del = len(df2[df2.VidInfo == vid]) - leave
    drop = list(np.random.choice(df2[df2.VidInfo == vid].index, to_del, replace=False))
    df2.drop(drop, inplace=True)

len(df2)

1503

In [39]:
df2.VidInfo.value_counts()

VidInfo
vid1    167
vid2    167
vid3    167
vid4    167
vid5    167
yt1     167
yt2     167
yt5     167
yt3     167
Name: count, dtype: int64

In [41]:
df2[df2.VidInfo == 'vid3'].tail(20)

,VidInfo,Action,StartFrame,EndFrame,Len
921,vid3,none,87266,87365,100
924,vid3,none,87838,87937,100
927,vid3,none,88144,88243,100
930,vid3,none,88959,89058,100
934,vid3,none,94092,94191,100
935,vid3,none,94194,94293,100
937,vid3,none,94468,94567,100
938,vid3,none,94736,94835,100
940,vid3,none,94940,95039,100
941,vid3,none,95042,95141,100


In [42]:
df.Action.value_counts()

Action
pass      1500
tackle     248
goal       199
save       154
Name: count, dtype: int64

In [43]:
df3 = pd.concat([df, df2], ignore_index=True)
df3

,VidInfo,Action,StartFrame,EndFrame,Len
0,vid1,pass,347,370,24
1,vid1,goal,583,654,72
2,vid1,pass,1051,1096,46
3,vid1,pass,2748,2796,49
4,vid1,save,2972,2998,27
...,...,...,...,...,...
3599,yt3,none,67633,67732,100
3600,yt3,none,67735,67834,100
3601,yt3,none,68080,68179,100
3602,yt3,none,70433,70532,100


In [44]:
df3.Action.value_counts()

Action
none      1503
pass      1500
tackle     248
goal       199
save       154
Name: count, dtype: int64

In [45]:
df3.VidInfo.value_counts()

VidInfo
yt5     612
yt2     578
vid4    476
yt1     406
yt3     367
vid5    337
vid2    306
vid1    262
vid3    260
Name: count, dtype: int64

In [47]:
df3.head()

,VidInfo,Action,StartFrame,EndFrame,Len
0,vid1,pass,347,370,24
1,vid1,goal,583,654,72
2,vid1,pass,1051,1096,46
3,vid1,pass,2748,2796,49
4,vid1,save,2972,2998,27


In [43]:
import os

num = 0
dir = os.path.abspath('/home/kyrylo/dev/clips/clips_dataset')
lst = ['goal', 'none', 'pass', 'save', 'tackle']
for el in lst:
    print(f"{len(os.listdir(os.path.join(dir, el)))}-{el}")
    num += len(os.listdir(os.path.join(dir, el)))
print(f"total length: {num}")

7960-goal
7500-none
7500-pass
7700-save
8680-tackle
total length: 39340


In [91]:
import os

dir = os.path.abspath('/home/kyrylo/dev/clips/clips_dataset')
lst = ['goal', 'none', 'pass', 'save', 'tackle']
X = []
y = []

for action_dir in lst:
    path_to_vid = os.path.join(dir, action_dir)
    for filename in os.listdir(path_to_vid):
        X.append(f"{dir.split('/')[-1]}/{action_dir}/{filename}")
        y.append(action_dir)

print(len(X), len(y))

39340 39340


In [92]:
X[24001], y[24001]

('clips_dataset/save/vid4_save_id0459_41240-41339_5.mp4', 'save')

In [93]:
num = 0
for i in range(len(X)):
    if X[i].split("/")[1] == y[i]:
        num += 1
print(num)

39340


In [95]:
import pickle
import os

dir = os.path.join(os.getcwd(), 'action_recognition')

with open(f"{dir}/X.pickle", "wb") as f:
    pickle.dump(X, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(f"{dir}/y.pickle", "wb") as f:
    pickle.dump(y, f, protocol=pickle.HIGHEST_PROTOCOL)


In [96]:
with open(f"{dir}/X.pickle", "rb") as f:
    xnew = pickle.load(f)

with open(f"{dir}/y.pickle", "rb") as f:
    ynew = pickle.load(f)

In [97]:
print(len(xnew), len(ynew))

39340 39340


In [98]:
num = 0
for i in range(len(xnew)):
    if xnew[i].split('/')[1] == ynew[i]:
        num += 1
num

39340

In [100]:
ynew[7]

'goal'

In [102]:
print(len(X))
print(len(y))

39340
39340


In [104]:
dict_ = {"pass": 0, "goal": 1, "save": 2, "tackle": 3, "none": 4}
for i in range(len(y)):
    label = y[i]
    y[i] = dict_[label]
y[:5]

[1, 1, 1, 1, 1]

In [105]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True)

In [106]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

35406
3934
35406
3934


In [110]:
dict_from = {0: "pass", 1: "goal", 2: "save", 3: "tackle", 4: "none"}
for i in range(10):
    print(f'{X_train[i].split("/")[1]}-{dict_from[y_train[i]]}')

tackle-tackle
goal-goal
pass-pass
goal-goal
none-none
pass-pass
tackle-tackle
none-none
goal-goal
pass-pass


In [111]:
X_train[1]

'clips_dataset/goal/youtube_5_goal_id1468_4185-4284_0.mp4'

In [122]:
import os

dir = "/home/kyrylo/dev/clips"
with open('train.txt', 'w') as f:
    for i in range(len(X_train)):
        f.write(f"{os.path.join(dir, X_train[i])} {y_train[i]}\n")

with open('val.txt', 'w') as f:
    for i in range(len(X_test)):
        f.write(f"{os.path.join(dir, X_test[i])} {y_test[i]}\n")

In [119]:
with open("clips/custom_data/val.txt", "r") as read_file:
        lines_train = read_file.readlines()

lines_train = [l[:-1] for l in lines_train]
len(lines_train)

3934

In [120]:
train_Xs = {l.split(" ")[0]: int(l.split(" ")[1]) for l in lines_train}
#val_Xs = {l.split(" ")[0]: int(l.split(" ")[1]) for l in lines_val}

# Recalculate weights for new splits

# For train split
y_new = list(train_Xs.values())
y_set = set(y_new)

class_counts = {k: 0 for k in list(y_set)}
for label in y_new:
        class_counts[label] += 1
class_counts

{0: 781, 1: 829, 2: 746, 3: 830, 4: 748}

In [ ]:
d = {0: 6719, 1: 7131, 2: 6954, 3: 7850, 4: 6752}
d2 = {0: 781, 1: 829, 2: 746, 3: 830, 4: 748}
